In [565]:
import torch
from torch import nn,optim
import torch.nn.functional
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os as os
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,multilabel_confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc,roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from scipy.stats import kurtosis,skew
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif
import scipy.stats
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel

import pickle
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE

#import warnings

#warnings.simplefilter(action='ignore', category=FutureWarning)

In [566]:
path_dataframes="/Users/alexdediu/git-projects/Data-Challenge-PHM-Asia-Pacific-2023/dataframes"


In [567]:
df_train = pd.read_csv(f"{path_dataframes}/df_train.csv")
# df_test = pd.read_csv("/Users/alexdediu/git-projects/Data-Challenge-PHM-Asia-Pacific-2023/df_test.csv")

df_labels = pd.read_csv(f"{path_dataframes}/df_labels_train.csv")


In [568]:
df_train 

,index,P1_min,P5_HarmonicAmplitudeRatio,P5_rms,P5_variance,P5_std,P1_median,P1_clearance_factor,P5_min,P1_variance,...,P1_rms,P1_HarmonicAmplitudeRatio,P2_75th_percentile,P2_variance,P2_rms,P2_std,P5_25th_percentile,P2_min,P6_HarmonicAmplitudeRatio,P5_fft_25th_percentile
0,0,0.151379,0.404716,0.908139,0.989111,0.990348,0.070910,0.835385,0.002855,0.997566,...,0.957028,0.500515,0.793864,0.742221,0.587077,0.764873,0.173932,0.014342,0.535152,0.113235
1,1,0.151707,0.386983,0.957102,0.970816,0.974082,0.103330,0.856063,0.002856,0.999072,...,0.970137,0.496295,0.762330,0.741918,0.572730,0.764588,0.177174,0.014171,0.534254,0.159739
2,2,0.152334,0.385712,0.908687,0.942070,0.948400,0.227248,0.884633,0.002595,0.997855,...,0.979246,0.492228,0.765295,0.752047,0.614115,0.774074,0.180487,0.014027,0.516265,0.047297
3,3,0.152786,0.384052,0.937148,0.964525,0.968475,0.163223,0.884439,0.002830,0.995734,...,0.969917,0.492075,0.775176,0.751956,0.622608,0.773988,0.181395,0.014103,0.509841,0.081392
4,4,0.152716,0.382267,0.961120,0.975974,0.978674,0.144362,0.873784,0.003023,0.996313,...,0.974559,0.490276,0.792718,0.745000,0.595336,0.767477,0.194015,0.014159,0.505803,0.088930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,172,0.339054,1.000000,0.205255,0.504254,0.536422,0.048881,0.598945,0.005946,0.585004,...,0.245719,1.000000,0.593336,1.000000,1.000000,1.000000,0.259250,0.087961,0.687747,0.361840
173,173,0.824787,0.697196,0.719323,0.781425,0.801969,0.509144,0.428516,0.880071,0.583065,...,0.138386,0.717433,0.569406,0.647391,0.448477,0.675020,0.221795,0.040686,0.730663,0.504753
174,174,0.301860,0.438246,0.344787,0.577233,0.608061,0.772911,0.737837,0.592579,0.531799,...,0.024519,0.362797,0.000000,0.576285,0.348801,0.606348,0.218800,0.026831,0.561968,0.464898
175,175,0.402989,0.000000,0.204130,0.497790,0.530013,1.000000,0.636050,0.440788,0.500953,...,0.139861,0.000000,0.127847,0.569685,0.239404,0.599916,0.219502,0.024886,0.000000,0.355554


In [569]:
df_labels = df_labels[106:153]

In [570]:
df_labels.head()

,index,Case,Spacecraft,Condition,SV1,SV2,SV3,SV4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,BV1,binary_condition
106,106,107,1,2,25,100,100,100,0,0,0,0,0,0,0,0,1
107,107,108,1,2,50,100,100,100,0,0,0,0,0,0,0,0,1
108,108,109,1,2,75,100,100,100,0,0,0,0,0,0,0,0,1
109,109,110,1,2,100,0,100,100,0,0,0,0,0,0,0,0,1
110,110,111,1,2,100,25,100,100,0,0,0,0,0,0,0,0,1


In [571]:
column_to_number = {
    'SV1': 1,
    'SV2': 2,
    'SV3': 3,
    'SV4': 4,
}

df_labels['fault_sv'] = 0

for index, row in df_labels.iterrows():
    for col in column_to_number.keys():
        if row[col] < 100:
            df_labels.at[index, 'fault_sv'] = column_to_number[col]
            break  



In [572]:
# X = df_train
# y = df_labels['anomaly_location']

# smote = SMOTE(k_neighbors=2,random_state=42)

# X_resampled, y_resampled = smote.fit_resample(X, y)


# df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
# df_resampled['target'] = y_resampled

In [573]:
# import plotly.graph_objs as go
# import plotly.io as pio

# # Supponendo che tu abbia già importato i DataFrame df_train e df_labels
# # df_train contiene i dati per lo scatter plot
# # df_labels contiene la colonna "anomaly_location" che vuoi utilizzare per colorare i punti dello scatter plot

# # Lista dei valori unici nella colonna "anomaly_location"
# unique_labels = df_labels["anomaly_location"].unique()

# # Lista di tracciamenti per ogni valore unico
# traces = []

# for label in unique_labels:
#     # Seleziona i punti corrispondenti a ciascun valore unico
#     df_subset = df_train[df_labels["anomaly_location"] == label]
    
#     # Crea un tracciamento per i punti con il colore corrispondente e il nome della legenda
#     trace = go.Scatter(x=df_subset.index, y=df_subset["P2_variance"], 
#                        mode='markers', 
#                        marker=dict(color=label),
#                        name=str(label))
    
#     # Aggiungi il tracciamento alla lista di tracciamenti
#     traces.append(trace)

# # Aggiungi un secondo scatter plot con tutti i dati
# scatter_plot = go.Scatter(x=df_train.index, y=df_train["P1_variance"], 
#                           mode='markers', 
#                           marker=dict(color='black', size=5), # Aggiungi la dimensione 5 per il secondo scatter plot
#                           name='P1_variance')

# # Aggiungi il secondo scatter plot alla lista di tracciamenti
# traces.append(scatter_plot)

# # Layout del grafico con legenda
# layout = go.Layout(title="P1_variance - Anomalie", 
#                    xaxis=dict(title="Indice"), 
#                    yaxis=dict(title="P1_variance"),
#                    showlegend=True)

# # Crea la figura e visualizzala
# fig = go.Figure(data=traces, layout=layout)
# pio.show(fig)


In [574]:
df_train

,index,P1_min,P5_HarmonicAmplitudeRatio,P5_rms,P5_variance,P5_std,P1_median,P1_clearance_factor,P5_min,P1_variance,...,P1_rms,P1_HarmonicAmplitudeRatio,P2_75th_percentile,P2_variance,P2_rms,P2_std,P5_25th_percentile,P2_min,P6_HarmonicAmplitudeRatio,P5_fft_25th_percentile
0,0,0.151379,0.404716,0.908139,0.989111,0.990348,0.070910,0.835385,0.002855,0.997566,...,0.957028,0.500515,0.793864,0.742221,0.587077,0.764873,0.173932,0.014342,0.535152,0.113235
1,1,0.151707,0.386983,0.957102,0.970816,0.974082,0.103330,0.856063,0.002856,0.999072,...,0.970137,0.496295,0.762330,0.741918,0.572730,0.764588,0.177174,0.014171,0.534254,0.159739
2,2,0.152334,0.385712,0.908687,0.942070,0.948400,0.227248,0.884633,0.002595,0.997855,...,0.979246,0.492228,0.765295,0.752047,0.614115,0.774074,0.180487,0.014027,0.516265,0.047297
3,3,0.152786,0.384052,0.937148,0.964525,0.968475,0.163223,0.884439,0.002830,0.995734,...,0.969917,0.492075,0.775176,0.751956,0.622608,0.773988,0.181395,0.014103,0.509841,0.081392
4,4,0.152716,0.382267,0.961120,0.975974,0.978674,0.144362,0.873784,0.003023,0.996313,...,0.974559,0.490276,0.792718,0.745000,0.595336,0.767477,0.194015,0.014159,0.505803,0.088930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,172,0.339054,1.000000,0.205255,0.504254,0.536422,0.048881,0.598945,0.005946,0.585004,...,0.245719,1.000000,0.593336,1.000000,1.000000,1.000000,0.259250,0.087961,0.687747,0.361840
173,173,0.824787,0.697196,0.719323,0.781425,0.801969,0.509144,0.428516,0.880071,0.583065,...,0.138386,0.717433,0.569406,0.647391,0.448477,0.675020,0.221795,0.040686,0.730663,0.504753
174,174,0.301860,0.438246,0.344787,0.577233,0.608061,0.772911,0.737837,0.592579,0.531799,...,0.024519,0.362797,0.000000,0.576285,0.348801,0.606348,0.218800,0.026831,0.561968,0.464898
175,175,0.402989,0.000000,0.204130,0.497790,0.530013,1.000000,0.636050,0.440788,0.500953,...,0.139861,0.000000,0.127847,0.569685,0.239404,0.599916,0.219502,0.024886,0.000000,0.355554


In [575]:
df_train

,index,P1_min,P5_HarmonicAmplitudeRatio,P5_rms,P5_variance,P5_std,P1_median,P1_clearance_factor,P5_min,P1_variance,...,P1_rms,P1_HarmonicAmplitudeRatio,P2_75th_percentile,P2_variance,P2_rms,P2_std,P5_25th_percentile,P2_min,P6_HarmonicAmplitudeRatio,P5_fft_25th_percentile
0,0,0.151379,0.404716,0.908139,0.989111,0.990348,0.070910,0.835385,0.002855,0.997566,...,0.957028,0.500515,0.793864,0.742221,0.587077,0.764873,0.173932,0.014342,0.535152,0.113235
1,1,0.151707,0.386983,0.957102,0.970816,0.974082,0.103330,0.856063,0.002856,0.999072,...,0.970137,0.496295,0.762330,0.741918,0.572730,0.764588,0.177174,0.014171,0.534254,0.159739
2,2,0.152334,0.385712,0.908687,0.942070,0.948400,0.227248,0.884633,0.002595,0.997855,...,0.979246,0.492228,0.765295,0.752047,0.614115,0.774074,0.180487,0.014027,0.516265,0.047297
3,3,0.152786,0.384052,0.937148,0.964525,0.968475,0.163223,0.884439,0.002830,0.995734,...,0.969917,0.492075,0.775176,0.751956,0.622608,0.773988,0.181395,0.014103,0.509841,0.081392
4,4,0.152716,0.382267,0.961120,0.975974,0.978674,0.144362,0.873784,0.003023,0.996313,...,0.974559,0.490276,0.792718,0.745000,0.595336,0.767477,0.194015,0.014159,0.505803,0.088930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,172,0.339054,1.000000,0.205255,0.504254,0.536422,0.048881,0.598945,0.005946,0.585004,...,0.245719,1.000000,0.593336,1.000000,1.000000,1.000000,0.259250,0.087961,0.687747,0.361840
173,173,0.824787,0.697196,0.719323,0.781425,0.801969,0.509144,0.428516,0.880071,0.583065,...,0.138386,0.717433,0.569406,0.647391,0.448477,0.675020,0.221795,0.040686,0.730663,0.504753
174,174,0.301860,0.438246,0.344787,0.577233,0.608061,0.772911,0.737837,0.592579,0.531799,...,0.024519,0.362797,0.000000,0.576285,0.348801,0.606348,0.218800,0.026831,0.561968,0.464898
175,175,0.402989,0.000000,0.204130,0.497790,0.530013,1.000000,0.636050,0.440788,0.500953,...,0.139861,0.000000,0.127847,0.569685,0.239404,0.599916,0.219502,0.024886,0.000000,0.355554


In [576]:
# df_labels = df_labels[df_labels["anomaly_location"] != 0]

In [577]:
# df_labels = df_labels[153:]

df_train = df_train[106:153]

In [578]:
df_labels

,index,Case,Spacecraft,Condition,SV1,SV2,SV3,SV4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,BV1,binary_condition,fault_sv
106,106,107,1,2,25,100,100,100,0,0,0,0,0,0,0,0,1,1
107,107,108,1,2,50,100,100,100,0,0,0,0,0,0,0,0,1,1
108,108,109,1,2,75,100,100,100,0,0,0,0,0,0,0,0,1,1
109,109,110,1,2,100,0,100,100,0,0,0,0,0,0,0,0,1,2
110,110,111,1,2,100,25,100,100,0,0,0,0,0,0,0,0,1,2
111,111,112,1,2,100,50,100,100,0,0,0,0,0,0,0,0,1,2
112,112,113,1,2,100,75,100,100,0,0,0,0,0,0,0,0,1,2
113,113,114,1,2,100,100,0,100,0,0,0,0,0,0,0,0,1,3
114,114,115,1,2,100,100,25,100,0,0,0,0,0,0,0,0,1,3
115,115,116,1,2,100,100,50,100,0,0,0,0,0,0,0,0,1,3


In [579]:
df_train

,index,P1_min,P5_HarmonicAmplitudeRatio,P5_rms,P5_variance,P5_std,P1_median,P1_clearance_factor,P5_min,P1_variance,...,P1_rms,P1_HarmonicAmplitudeRatio,P2_75th_percentile,P2_variance,P2_rms,P2_std,P5_25th_percentile,P2_min,P6_HarmonicAmplitudeRatio,P5_fft_25th_percentile
106,106,0.000000,0.432084,0.920328,0.972287,0.975392,0.144575,0.829670,0.017883,1.000000,...,0.952962,0.517702,0.744475,0.806982,0.511060,0.825154,0.181802,0.000403,0.581795,0.290664
107,107,0.079907,0.401804,0.882183,0.958276,0.962897,0.144355,0.858157,0.007345,0.997689,...,0.955945,0.505588,0.711381,0.671879,0.718494,0.698408,0.164252,0.004243,0.549119,0.191246
108,108,0.127828,0.392466,0.945762,0.972719,0.975777,0.212011,0.870260,0.004342,0.996105,...,0.955908,0.497703,0.731032,0.731195,0.626936,0.754523,0.176166,0.006799,0.521787,0.133184
109,109,0.818104,0.608123,0.429840,0.321349,0.350809,0.496424,0.182820,0.002761,0.164377,...,0.456168,0.868375,0.419443,0.005956,0.230272,0.006808,0.744361,0.424390,0.716813,0.220123
110,110,0.181181,0.401224,1.000000,1.000000,1.000000,0.276834,0.725711,0.003959,0.963960,...,0.896901,0.516734,0.645957,0.687598,0.645089,0.713352,0.163519,0.008433,0.557830,0.000000
111,111,0.173570,0.389152,0.932861,0.971708,0.974876,0.109605,0.795423,0.003394,0.982280,...,0.941770,0.505068,0.685928,0.759736,0.528471,0.781261,0.178516,0.013025,0.529151,0.001112
112,112,0.163054,0.400904,0.916673,0.989555,0.990742,0.051578,0.827589,0.003027,0.992054,...,0.963407,0.500562,0.779643,0.736040,0.571082,0.759074,0.174408,0.013710,0.546460,0.125076
113,113,0.876307,0.749147,0.250187,0.114248,0.128759,0.195464,0.113976,0.483047,0.184946,...,0.481699,0.776817,0.519733,0.070744,0.000000,0.080016,0.970227,0.179641,0.808542,0.807135
114,114,0.179239,0.425615,0.891330,0.935796,0.942775,0.273140,0.836484,0.019392,0.965570,...,0.934751,0.512463,0.813806,0.744205,0.709016,0.766732,0.167988,0.013892,0.625088,0.040945
115,115,0.153747,0.404177,0.948455,0.954336,0.959378,0.140854,0.866810,0.047007,0.985524,...,0.951909,0.502909,0.774071,0.741056,0.616564,0.763780,0.177433,0.013523,0.660065,0.369574


In [580]:
df_labels

,index,Case,Spacecraft,Condition,SV1,SV2,SV3,SV4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,BV1,binary_condition,fault_sv
106,106,107,1,2,25,100,100,100,0,0,0,0,0,0,0,0,1,1
107,107,108,1,2,50,100,100,100,0,0,0,0,0,0,0,0,1,1
108,108,109,1,2,75,100,100,100,0,0,0,0,0,0,0,0,1,1
109,109,110,1,2,100,0,100,100,0,0,0,0,0,0,0,0,1,2
110,110,111,1,2,100,25,100,100,0,0,0,0,0,0,0,0,1,2
111,111,112,1,2,100,50,100,100,0,0,0,0,0,0,0,0,1,2
112,112,113,1,2,100,75,100,100,0,0,0,0,0,0,0,0,1,2
113,113,114,1,2,100,100,0,100,0,0,0,0,0,0,0,0,1,3
114,114,115,1,2,100,100,25,100,0,0,0,0,0,0,0,0,1,3
115,115,116,1,2,100,100,50,100,0,0,0,0,0,0,0,0,1,3


In [581]:
# X_train, X_test, y_train, y_test = train_test_split(df_train, df_labels["anomaly_location"], test_size=0.33, random_state=42)

# random_forest = RandomForestClassifier(n_estimators=50, random_state=42)

# random_forest.fit(X_train, y_train)

# predictions = random_forest.predict(X_test)

# accuracy = accuracy_score(y_test, predictions)
# print("Accuracy:", accuracy)

# conf_matrix = confusion_matrix(y_test, predictions)
# print("Confusion Matrix:")
# print(conf_matrix)

# class_report = classification_report(y_test, predictions)
# print("Classification Report:")
# print(class_report)


In [582]:
# Assuming df_train is your input DataFrame and df_labels["anomaly_location"] is your target variable
# Assuming you have already defined X_train and y_train

# Get the number of unique locations
num_locations = df_labels["fault_sv"].unique()
print(num_locations)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_train, df_labels["fault_sv"], test_size=0.01,random_state=123)

[1 2 3 4]


In [583]:
y_train

117    4
150    4
124    1
149    4
137    1
148    3
116    3
111    2
107    1
112    2
132    3
119    4
118    4
130    3
142    2
127    2
133    4
121    1
145    3
113    3
126    2
152    4
109    2
129    3
147    3
136    4
122    1
106    1
110    2
135    4
120    4
146    3
131    3
141    2
143    2
115    3
138    1
139    1
128    2
125    2
123    1
144    2
140    1
134    4
108    1
151    4
Name: fault_sv, dtype: int64

In [584]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
import numpy as np

confusion_matrices = []
models = []

for location in num_locations:
    y_binary_train = (y_train == location).astype(int)
    
    model = RandomForestClassifier(n_estimators=150,random_state=123)
    model.fit(X_train, y_binary_train)
    
    models.append(model)
    
    y_pred_binary = model.predict(X_test)
    
    cm = confusion_matrix(y_binary_train, model.predict(X_train))
    confusion_matrices.append(cm)

predictions = []
for model in models:
    pred = model.predict(X_test)
    print(pred)
    print(y_test)
    predictions.append(pred)



[0]
114    3
Name: fault_sv, dtype: int64
[0]
114    3
Name: fault_sv, dtype: int64
[0]
114    3
Name: fault_sv, dtype: int64
[0]
114    3
Name: fault_sv, dtype: int64


In [585]:
df_sv_test = pd.read_csv(f"{path_dataframes}/df_test_sv.csv")
# df_bubble_test = df_bubble_test.drop(columns="Unnamed: 0")

In [586]:
predictions = []
for model in models:
    pred = model.predict(df_sv_test)
    
    predictions.append(pred)
predictions

[array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 1, 0, 0]),
 array([0, 0, 1, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0])]

In [587]:
df_sv_test

,index,P1_min,P5_HarmonicAmplitudeRatio,P5_rms,P5_variance,P5_std,P1_median,P1_clearance_factor,P5_min,P1_variance,...,P1_rms,P1_HarmonicAmplitudeRatio,P2_75th_percentile,P2_variance,P2_rms,P2_std,P5_25th_percentile,P2_min,P6_HarmonicAmplitudeRatio,P5_fft_25th_percentile
0,179,0.182168,0.428010,0.980064,0.987083,0.988548,0.193701,0.729427,0.003761,0.960428,...,0.902002,0.524429,0.797330,0.679828,0.676689,0.705973,0.174802,0.007976,0.588214,0.018881
1,181,0.162786,0.396816,0.930225,0.928183,0.935939,0.047864,0.868178,0.018448,0.995939,...,0.976223,0.494610,0.775503,0.750242,0.608319,0.772385,0.169909,0.014086,0.519924,0.277822
2,190,0.155755,0.431533,0.940529,0.933203,0.940448,0.164763,0.867371,0.042128,0.985052,...,0.947702,0.505549,0.770602,0.748659,0.619560,0.770904,0.174555,0.013683,0.701148,0.270207
3,202,0.167160,0.339856,0.812565,0.870523,0.883807,0.016741,0.990733,0.009720,0.906205,...,0.785012,0.449199,0.748350,0.858009,0.811091,0.872061,0.157568,0.013006,0.480677,0.392654
4,205,0.165093,0.331766,0.876174,0.908170,0.917917,-0.007509,1.002227,0.003162,0.921515,...,0.837375,0.433291,0.741739,0.867685,0.778800,0.880899,0.157479,0.012307,0.457243,0.231810
5,211,0.159077,0.329359,0.889466,0.916846,0.925739,0.000314,1.001944,0.003408,0.925780,...,0.811674,0.433105,0.745907,0.836292,0.775557,0.852160,0.160581,0.012957,0.458831,0.284059
6,212,0.174721,0.347295,0.816773,0.893796,0.904926,0.016760,1.030072,0.002906,0.921096,...,0.880597,0.440190,0.707148,0.857201,0.741910,0.871323,0.149474,0.012710,0.451124,0.124611
7,214,0.250613,0.325654,0.824912,0.866553,0.880194,0.079759,0.878126,0.086240,0.874428,...,0.685379,0.433933,0.759952,0.824476,0.659197,0.841293,0.177092,0.013212,0.421845,0.228352
8,220,0.162941,0.328731,0.861372,0.905859,0.915830,0.002774,0.992679,0.003270,0.918470,...,0.838425,0.428439,0.747336,0.863441,0.833745,0.877025,0.158852,0.013035,0.460249,0.245187
